In [2]:
import numpy as np
import scipy.misc
from keras import backend as K
# from keras import initializations
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Building Katakana Model with CNN

In [3]:
nb_classes = 46
# input image dimensions
img_rows, img_cols = 32, 32

ary = np.load("regular_katakana.npz")['arr_0'].reshape([-1, 63, 64]).astype(np.float32) / 15
X_train = np.zeros([nb_classes * 1410, img_rows, img_cols], dtype=np.float32)
for i in range(nb_classes * 1410):
    X_train[i] = scipy.misc.imresize(ary[i], (img_rows, img_cols), mode='F')
    # X_train[i] = ary[i]
Y_train = np.repeat(np.arange(nb_classes), 1410)

X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2)

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# convert class vectors to categorical matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

datagen = ImageDataGenerator(rotation_range=15, zoom_range=0.20)
datagen.fit(X_train)

model = Sequential()

def m6_1():
    model.add(Conv2D(32, 3, 3, input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        18496     
__________

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 1...)`
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [4]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=16), samples_per_epoch=X_train.shape[0],
                    nb_epoch=30, validation_data=(X_test, Y_test))

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  app.launch_new_instance()
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=(array([[[..., steps_per_epoch=3243, epochs=30)`
  app.launch_new_instance()


Epoch 1/30
3243/3243 [==============================] - 194s 60ms/step - loss: 1.9425 - acc: 0.4623 - val_loss: 0.4934 - val_acc: 0.8989
Epoch 2/30
3243/3243 [==============================] - 199s 61ms/step - loss: 0.8027 - acc: 0.7916 - val_loss: 0.3703 - val_acc: 0.9384
Epoch 3/30
3243/3243 [==============================] - 184s 57ms/step - loss: 0.6178 - acc: 0.8530 - val_loss: 0.3203 - val_acc: 0.9507 loss: 0.6197 - acc: 0.852 - ET - ETA: 3s - loss: 0.6191 - 
Epoch 4/30
3243/3243 [==============================] - 188s 58ms/step - loss: 0.5426 - acc: 0.8773 - val_loss: 0.3677 - val_acc: 0.9287
Epoch 5/30
3243/3243 [==============================] - 194s 60ms/step - loss: 0.5088 - acc: 0.8865 - val_loss: 0.2803 - val_acc: 0.9607
Epoch 6/30
3243/3243 [==============================] - 190s 58ms/step - loss: 0.4936 - acc: 0.8939 - val_loss: 0.3289 - val_acc: 0.9449
Epoch 7/30
3243/3243 [==============================] - 191s 59ms/step - loss: 0.4832 - acc: 0.8971 - val_loss: 0.2701 

## Remove dropouts after the training is done

In [5]:
import keras
for k in model.layers:
    if type(k) is keras.layers.Dropout:
        model.layers.remove(k)

In [7]:
model.save('katakanaModel.h5')